In [ ]:
#init
import torch, numpy, sys
sys.path.append("..")
from chatbot_end2end.model.supervised import Supervised
from nlptools.utils import Config

cfg = Config("../config/cornell.yml")
model = Supervised.build(cfg)

In [ ]:
model.skill.model.config

In [ ]:
dialog = iter(model.generator).next()
dialog.to("cuda:0")
response = dialog["response_cornell"].data
response_mask = dialog["response_mask_cornell"].data
utterance = dialog["utterance"].data
utterance_mask = dialog["utterance_mask"].data
sentiment = dialog["sentiment"].data
print(utterance, utterance.shape)
print(utterance_mask, utterance_mask.shape)
print(response, response.shape)
print(response_mask, response_mask.shape)
print(sentiment[:10, :], sentiment.shape)

In [ ]:
encoder_out, encoder_hidden = model.skill.model.dialog_embedding(utterance, utterance_mask, sentiment)
prev_output = response.data[:, :-1]
target_output = response.data[:, 1:]
print(encoder_out.shape)
print(prev_output.shape)
print(target_output.shape)

In [ ]:
for parameter in model.skill.model.decoder.fc3.parameters():
    print(parameter.requires_grad)

In [ ]:
target_output = target_output.unsqueeze(-1).contiguous().view(-1)
target_output.shape

In [ ]:
#import nlptools.zoo.encoders.transformer
import nlptools.zoo.encoders.lstm
import nlptools.zoo.encoders.attention
import importlib
#importlib.reload(nlptools.zoo.encoders.transformer)
importlib.reload(nlptools.zoo.encoders.lstm)
importlib.reload(nlptools.zoo.encoders.attention)
#from nlptools.zoo.encoders.transformer import TransformerDecoder, TransformerEncoder
from nlptools.zoo.encoders.lstm import LSTMDecoder, LSTMEncoder
from nlptools.zoo.encoders.attention import MultiheadAttention

embedding = model.skill.model.encoder.embedding
decoder_hidden_layers = 1
decoder_attention_heads = 3
decoder_hidden_size = 2048
dropout = 0
# decoder = TransformerDecoder(embedding, decoder_hidden_layers, decoder_attention_heads, decoder_hidden_size, dropout=dropout)
decoder = LSTMDecoder(embedding, decoder_hidden_layers, decoder_hidden_size, dropout=dropout)
decoder = decoder.to("cuda:0")
output = decoder(prev_output, encoder_out, utterance_mask, encoder_hidden)
output.shape

In [ ]:
decoder